In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
%run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_syllable_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
save_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', X_train, y_train, other_features = X_other_features_train)
save_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', X_test, y_test, other_features = X_other_features_test)
save_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', other_features=True)

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
CREATING SYLLABLE DICTIONARY...
CREATION SUCCESSFUL!
GENERATING X AND y...
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [77]:
%run ../../../prepare_data.py
gen = generate_fake_epoch_syllables(X_train[:20], X_other_features_train[:20], y_train[:20], 16, 5168)

In [85]:
# prints test word from generator
# print(X_train[0])
# print(syllable_dictionary[test[0][0][0][0].tolist().index(1)])
print(np.array(test[0][0]).shape)
for el in test[0][0][0]:
    print(el)
    print(syllable_dictionary[el.tolist().index(1)])

(4, 11, 5168)
[0 0 0 ..., 0 0 0]
bolj
[0 0 0 ..., 0 0 0]
ši
[0 0 0 ..., 0 0 0]
mi
[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]



In [84]:
# print(X_train[0])
test = next(gen)
# eye = np.eye(5168)[X_train[0]]
# print(X_train[0].tolist())
# np.eye(5168)[[2, 1]]



In [123]:
np.eye(10)[[3,7,2,5]]
print(eye[[2, 3]].shape)

(2, 5168)


In [11]:
X_validate.shape
# len(syllable_dictionary)

(54222, 11)

In [46]:
print(len(syllable_dictionary))
shuffle_vector_location = '../../internal_representations/inputs/shuffle_vector'
X_train, X_other_features_train, y_train = generate_syllable_X_and_y(syllable_dictionary, max_word, max_num_vowels, train_content, vowels, accetuated_vowels, feature_dictionary, shuffle_vector_location + '_train.h5')


5168
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
(430151, 11)
(430151,)
INPUTS SHUFFELED!


In [5]:
print (X_train.shape)
print (X_test.shape)
print (X_validate.shape)
# X_other_features = create_X_features(content)
# print (X_other_features[178200])
decode_position = 38
print (decode_input(X_validate[decode_position], dictionary))
# print (X_other_features_train[0])
print (decode_X_features(feature_dictionary, [X_other_features_train[decode_position]]))
# print (len(X_other_features_train[0]))

(430151, 23, 30)
(52058, 23, 30)
(54222, 23, 30)


NameError: name 'dictionary' is not defined

In [9]:
num_examples = len(X_train) # training set size
# nn_input_dim = max_word * len(dictionary) # input layer dimensionality
# nn_output_dim = max_num_vowels * max_num_vowels # output layer dimensionality
nn_output_dim = 11
nn_hdim = 516
batch_size = 16
actual_epoch = 20
num_fake_epoch = 20

# Gradient descent parameters (I picked these by hand)
# epsilon = 1 # learning rate for gradient descent
# reg_lambda = 1 # regularization strength

In [15]:
conv_input_shape=(11, 5168)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(44, (2), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(44, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)

In [16]:
model = Model(inputs=[conv_input, othr_input], outputs=x)

# epochs = 5
# lrate = 0.1
# decay = lrate/epochs
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Compile model
# keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [93]:
# history = model.fit_generator(generate_fake_epoch_syllables(X_test[:1000], X_other_features_test[:1000], y_test[:1000], batch_size), X_test[:1000].shape[0]/(batch_size * num_fake_epoch), epochs=actual_epoch*num_fake_epoch, validation_data=generate_fake_epoch_syllables(X_test[:1000], X_other_features_test[:1000], y_test[:1000], batch_size), validation_steps=X_test[:1000].shape[0]/(batch_size * num_fake_epoch))

In [17]:
# history = model.fit_generator(generate_fake_epoch(X_train, X_other_features_train, y_train, batch_size), X_train.shape[0]/(batch_size * num_fake_epoch), epochs=actual_epoch*num_fake_epoch, validation_data=([X_test, X_other_features_test], y_test))
history = model.fit_generator(generate_fake_epoch_syllables(X_train, X_other_features_train, y_train, batch_size), X_train.shape[0]/(batch_size * num_fake_epoch), epochs=actual_epoch*num_fake_epoch, validation_data=generate_fake_epoch_syllables(X_test, X_other_features_test, y_test, batch_size), validation_steps=X_test.shape[0]/(batch_size * num_fake_epoch))


# model.fit([X_train, X_other_features_train], y_train, validation_data=([X_validate, X_other_features_validate], y_validate), epochs=1, batch_size=16)

Epoch 1/400
1345/1344 [==============================] - 127s - loss: 0.2063 - actual_accuracy: 0.2405 - val_loss: 0.1341 - val_actual_accuracy: 0.5387
Epoch 2/400
1345/1344 [==============================] - 135s - loss: 0.1152 - actual_accuracy: 0.6118 - val_loss: 0.1010 - val_actual_accuracy: 0.7036
Epoch 3/400
1345/1344 [==============================] - 149s - loss: 0.0903 - actual_accuracy: 0.7352 - val_loss: 0.0872 - val_actual_accuracy: 0.7650
Epoch 4/400
1345/1344 [==============================] - 151s - loss: 0.0758 - actual_accuracy: 0.7860 - val_loss: 0.0840 - val_actual_accuracy: 0.7826
Epoch 5/400
1345/1344 [==============================] - 148s - loss: 0.0666 - actual_accuracy: 0.8159 - val_loss: 0.0830 - val_actual_accuracy: 0.7887
Epoch 6/400
1345/1344 [==============================] - 153s - loss: 0.0580 - actual_accuracy: 0.8421 - val_loss: 0.0803 - val_actual_accuracy: 0.8002
Epoch 7/400
1345/1344 [==============================] - 155s - loss: 0.0526 - actual_ac

1345/1344 [==============================] - 178s - loss: 0.0106 - actual_accuracy: 0.9751 - val_loss: 0.1373 - val_actual_accuracy: 0.8439
Epoch 55/400
1345/1344 [==============================] - 177s - loss: 0.0107 - actual_accuracy: 0.9750 - val_loss: 0.1302 - val_actual_accuracy: 0.8512
Epoch 56/400
1345/1344 [==============================] - 176s - loss: 0.0098 - actual_accuracy: 0.9781 - val_loss: 0.1539 - val_actual_accuracy: 0.8531
Epoch 57/400
1345/1344 [==============================] - 178s - loss: 0.0102 - actual_accuracy: 0.9766 - val_loss: 0.1583 - val_actual_accuracy: 0.8401
Epoch 58/400
1345/1344 [==============================] - 178s - loss: 0.0099 - actual_accuracy: 0.9782 - val_loss: 0.1500 - val_actual_accuracy: 0.8344
Epoch 59/400
1345/1344 [==============================] - 176s - loss: 0.0100 - actual_accuracy: 0.9763 - val_loss: 0.1463 - val_actual_accuracy: 0.8551
Epoch 60/400
1345/1344 [==============================] - 179s - loss: 0.0105 - actual_accuracy

1345/1344 [==============================] - 191s - loss: 0.0061 - actual_accuracy: 0.9867 - val_loss: 0.1811 - val_actual_accuracy: 0.8409
Epoch 108/400
1345/1344 [==============================] - 204s - loss: 0.0054 - actual_accuracy: 0.9876 - val_loss: 0.1629 - val_actual_accuracy: 0.8543
Epoch 109/400
1345/1344 [==============================] - 201s - loss: 0.0054 - actual_accuracy: 0.9889 - val_loss: 0.1735 - val_actual_accuracy: 0.8501
Epoch 110/400
1345/1344 [==============================] - 184s - loss: 0.0063 - actual_accuracy: 0.9860 - val_loss: 0.1598 - val_actual_accuracy: 0.8470
Epoch 111/400
1345/1344 [==============================] - 188s - loss: 0.0055 - actual_accuracy: 0.9875 - val_loss: 0.1870 - val_actual_accuracy: 0.8436
Epoch 112/400
1345/1344 [==============================] - 184s - loss: 0.0057 - actual_accuracy: 0.9872 - val_loss: 0.1724 - val_actual_accuracy: 0.8570
Epoch 113/400
1345/1344 [==============================] - 192s - loss: 0.0051 - actual_ac

1345/1344 [==============================] - 191s - loss: 0.0043 - actual_accuracy: 0.9902 - val_loss: 0.1800 - val_actual_accuracy: 0.8528
Epoch 161/400
1345/1344 [==============================] - 191s - loss: 0.0039 - actual_accuracy: 0.9908 - val_loss: 0.2060 - val_actual_accuracy: 0.8382
Epoch 162/400
1345/1344 [==============================] - 189s - loss: 0.0044 - actual_accuracy: 0.9905 - val_loss: 0.2129 - val_actual_accuracy: 0.8505
Epoch 163/400
1345/1344 [==============================] - 194s - loss: 0.0042 - actual_accuracy: 0.9906 - val_loss: 0.1867 - val_actual_accuracy: 0.8524
Epoch 164/400
1345/1344 [==============================] - 192s - loss: 0.0042 - actual_accuracy: 0.9901 - val_loss: 0.1878 - val_actual_accuracy: 0.8562
Epoch 165/400
1345/1344 [==============================] - 190s - loss: 0.0039 - actual_accuracy: 0.9914 - val_loss: 0.1822 - val_actual_accuracy: 0.8528
Epoch 166/400
1345/1344 [==============================] - 193s - loss: 0.0040 - actual_ac

1345/1344 [==============================] - 197s - loss: 0.0029 - actual_accuracy: 0.9937 - val_loss: 0.1939 - val_actual_accuracy: 0.8566
Epoch 214/400
1345/1344 [==============================] - 195s - loss: 0.0037 - actual_accuracy: 0.9921 - val_loss: 0.2059 - val_actual_accuracy: 0.8489
Epoch 215/400
1345/1344 [==============================] - 196s - loss: 0.0030 - actual_accuracy: 0.9926 - val_loss: 0.1745 - val_actual_accuracy: 0.8547
Epoch 216/400
1345/1344 [==============================] - 198s - loss: 0.0030 - actual_accuracy: 0.9939 - val_loss: 0.1942 - val_actual_accuracy: 0.8574
Epoch 217/400
1345/1344 [==============================] - 197s - loss: 0.0032 - actual_accuracy: 0.9932 - val_loss: 0.1712 - val_actual_accuracy: 0.8551
Epoch 218/400
1345/1344 [==============================] - 198s - loss: 0.0032 - actual_accuracy: 0.9933 - val_loss: 0.2116 - val_actual_accuracy: 0.8344
Epoch 219/400
1345/1344 [==============================] - 197s - loss: 0.0031 - actual_ac

1345/1344 [==============================] - 200s - loss: 0.0025 - actual_accuracy: 0.9944 - val_loss: 0.2216 - val_actual_accuracy: 0.8390
Epoch 267/400
1345/1344 [==============================] - 199s - loss: 0.0027 - actual_accuracy: 0.9941 - val_loss: 0.2125 - val_actual_accuracy: 0.8478
Epoch 268/400
1345/1344 [==============================] - 198s - loss: 0.0022 - actual_accuracy: 0.9948 - val_loss: 0.1970 - val_actual_accuracy: 0.8558
Epoch 269/400
1345/1344 [==============================] - 198s - loss: 0.0024 - actual_accuracy: 0.9953 - val_loss: 0.1924 - val_actual_accuracy: 0.8593
Epoch 270/400
1345/1344 [==============================] - 199s - loss: 0.0031 - actual_accuracy: 0.9935 - val_loss: 0.2057 - val_actual_accuracy: 0.8520
Epoch 271/400
1345/1344 [==============================] - 196s - loss: 0.0029 - actual_accuracy: 0.9936 - val_loss: 0.1982 - val_actual_accuracy: 0.8597
Epoch 272/400
1345/1344 [==============================] - 200s - loss: 0.0026 - actual_ac

1345/1344 [==============================] - 197s - loss: 0.0025 - actual_accuracy: 0.9947 - val_loss: 0.2307 - val_actual_accuracy: 0.8405
Epoch 320/400
1345/1344 [==============================] - 193s - loss: 0.0025 - actual_accuracy: 0.9944 - val_loss: 0.2381 - val_actual_accuracy: 0.8432
Epoch 321/400
1345/1344 [==============================] - 199s - loss: 0.0021 - actual_accuracy: 0.9955 - val_loss: 0.2216 - val_actual_accuracy: 0.8563
Epoch 322/400
1345/1344 [==============================] - 203s - loss: 0.0023 - actual_accuracy: 0.9958 - val_loss: 0.2295 - val_actual_accuracy: 0.8359
Epoch 323/400
1345/1344 [==============================] - 202s - loss: 0.0016 - actual_accuracy: 0.9962 - val_loss: 0.2307 - val_actual_accuracy: 0.8516
Epoch 324/400
1345/1344 [==============================] - 198s - loss: 0.0024 - actual_accuracy: 0.9947 - val_loss: 0.2160 - val_actual_accuracy: 0.8512
Epoch 325/400
1345/1344 [==============================] - 199s - loss: 0.0021 - actual_ac

1345/1344 [==============================] - 199s - loss: 0.0021 - actual_accuracy: 0.9955 - val_loss: 0.2134 - val_actual_accuracy: 0.8508
Epoch 373/400
1345/1344 [==============================] - 205s - loss: 0.0018 - actual_accuracy: 0.9958 - val_loss: 0.2151 - val_actual_accuracy: 0.8597
Epoch 374/400
1345/1344 [==============================] - 203s - loss: 0.0023 - actual_accuracy: 0.9951 - val_loss: 0.2175 - val_actual_accuracy: 0.8516
Epoch 375/400
1345/1344 [==============================] - 203s - loss: 0.0023 - actual_accuracy: 0.9952 - val_loss: 0.2171 - val_actual_accuracy: 0.8439
Epoch 376/400
1345/1344 [==============================] - 201s - loss: 0.0017 - actual_accuracy: 0.9955 - val_loss: 0.2081 - val_actual_accuracy: 0.8535
Epoch 377/400
1345/1344 [==============================] - 209s - loss: 0.0016 - actual_accuracy: 0.9962 - val_loss: 0.2580 - val_actual_accuracy: 0.8462
Epoch 378/400
1345/1344 [==============================] - 198s - loss: 0.0024 - actual_ac

In [18]:
name = '20_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [46]:
dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
train_content, test_content, validate_content = split_content(content, 0.2, '../../internal_representations/inputs/content_shuffle_vector.h5')
feature_dictionary = create_feature_dictionary()
syllable_dictionary = create_syllables_dictionary(content, vowels)
def generate_input_from_word(word, max_word, dictionary):
    x = np.zeros((max_word, len(dictionary)))
    j = 0
    for c in list(word):
        index = 0
        for d in dictionary:
            if c == d:
                x[j, index] = 1
                break
            index += 1
        j += 1
    return x

# model = load_model()
# prediction = predict(model, generate_input_from_word('hidrija'))
# print decode_position(prediction[0])

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!


In [5]:
# create_syllables(content[375488][0], vowels)
create_syllables('izzvati', vowels)

['iz', 'zva', 'ti']

In [45]:

syllables_dictionary = create_syllables_dictionary(content, vowels)

NameError: name 'content' is not defined

In [9]:
syllables_dictionary = sorted(syllables_dictionary)
syllables_dictionary

['',
 '-o',
 'a',
 'ab',
 'ah',
 'akt',
 'al',
 'alf',
 'alg',
 'alt',
 'am',
 'an',
 'ant',
 'ar',
 'arh',
 'as',
 'at',
 'av',
 'ba',
 'bab',
 'bah',
 'baj',
 'bajt',
 'bak',
 'bal',
 'bam',
 'ban',
 'banj',
 'bank',
 'bar',
 'bark',
 'barv',
 'bas',
 'bast',
 'bat',
 'bav',
 'baz',
 'bač',
 'baš',
 'baž',
 'bde',
 'bdel',
 'bdet',
 'bdeč',
 'bdi',
 'bdim',
 'bdiš',
 'be',
 'bec',
 'bed',
 'beg',
 'beh',
 'bej',
 'bek',
 'bel',
 'belc',
 'belj',
 'belk',
 'bem',
 'ben',
 'benj',
 'benk',
 'bent',
 'ber',
 'bes',
 'bet',
 'bev',
 'bez',
 'beč',
 'beš',
 'bež',
 'bi',
 'bic',
 'bih',
 'bij',
 'bik',
 'bil',
 'bilj',
 'bilk',
 'bim',
 'bin',
 'bir',
 'bisk',
 'bit',
 'bitk',
 'biv',
 'bič',
 'biš',
 'bišč',
 'bja',
 'bjav',
 'bje',
 'bjej',
 'bjekt',
 'bjel',
 'bjem',
 'bjest',
 'bjet',
 'bješ',
 'bji',
 'bjih',
 'bjim',
 'bjo',
 'bju',
 'bla',
 'blag',
 'blah',
 'blak',
 'blam',
 'blan',
 'blast',
 'blat',
 'ble',
 'bled',
 'blej',
 'blek',
 'blem',
 'blesk',
 'blest',
 'blet',
 'bleč'

In [57]:
%run ../../../prepare_data.py
# generate_X_and_y(dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels, feature_dictionary)